# Metadata

**L1 Taxonomy** - Background Processes

**L2 Taxonomy** - Task Queues

**Subtopic** - Managing Task Priorities and Dependencies in Python Queues

**Use Case** - Implement a priority queue using heapq to manage task execution order based on dynamic priority levels.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt
Problem Statement:
- In the year 2125, you are the lead software engineer on the ISS Centurion, Earth’s most advanced space station.
- Aboard this orbital behemoth, a swarm of robotic assistants perform critical operations.
- These robots rely on an AI-powered mission manager to schedule tasks based on urgency.
- Your job is to implement the core of this mission manager — a dynamic task execution queue.
- Each task has a priority, and lower numbers indicate higher importance.
- Tasks with equal priority must be executed in the order they were received.
- Priorities can change over time due to critical updates or system decisions.
- Implement a class TaskQueue using heapq that supports the following:

Class Definition:
```python
class TaskQueue:
    def add_task(self, task_name: str, priority: int) -> None:
        '''
        Adds a task with a name and priority to the queue.
        Lower numbers mean higher priority.
        Tasks with the same priority are handled in FIFO order.
        If a task with the same name exists, its priority is updated.
        '''

    def get_next_task(self) -> str:
        '''
        Removes and returns the task with the highest priority.
        If multiple tasks have the same priority, returns the one added first.
        If the queue is empty, returns "NO TASKS".
        '''

    def peek_next_task(self) -> str:
        '''
        Returns the name of the task with the highest priority without removing it.
        If multiple tasks have the same priority, returns the one added first.
        If the queue is empty, returns "NO TASKS".
        '''
```

Input Format:
- The program receives a sequence of commands.
- Each command is one of:
 - "ADD <task_name> <priority>" — to add a task or update its priority.
 - "GET" — to retrieve the highest priority task.
 - "PEEK" — to view the highest priority task without removing it.

Input Constraints:
- task_name: non-empty string up to 100 characters.
- priority: integer between 1 and 1000000.
- Up to 100000 tasks may be added and 100000 commands processed.


Output Format:
- For each "GET" command, print the name of the task with the highest priority.
- For each "PEEK" command, print the name of the task with the highest priority.
- If there are multiple tasks with the same priority, return the one added or updated earliest.
- If no tasks are available during a "GET" or "PEEK", print "NO TASKS".

Example:
Input:
ADD email_processing 5
ADD data_backup 2
GET
ADD system_update 2
PEEK
ADD email_processing 1
GET
GET
PEEK

Example Output:
data_backup
system_update
email_processing
system_update
NO TASKS

# Requirements
Explicit and Implicit Requirements:
- Use heapq to implement the priority queue.
- Support four behaviors: add new task, update existing task’s priority, retrieve task, and peek at the next task.
- Tasks with the same name must be updated, not duplicated.
- "GET" and "PEEK" must always return the lowest-priority-number task.
- If the queue is empty on "GET" or "PEEK", print "NO TASKS".

Solution Expectations:
- Use a min-heap to manage tasks with tuples like (priority, insertion_order, task_name).
- Track a task index map (e.g., task_name -> (priority, insertion_order) or similar) to store the current valid state of each task.
- On "ADD", if the task exists:
 - The old entry in the heap should be logically invalidated (e.g., by ensuring get_next_task or peek_next_task can detect it's outdated).
 - A new entry with the updated priority and a new insertion_order (global counter) should be pushed onto the heap.
- On "GET" or "PEEK", repeatedly pop/peek from the heap until a valid task is found (i.e., its priority and insertion_order match what's stored in the task map). Invalid entries should be discarded during GET and implicitly handled during PEEK.
- Maintain FIFO order for tasks with the same priority using a monotonically increasing global counter for insertion_order.
- Ensure efficiency with heapq and a hashmap combined.


Edge Cases and Behaviour:
- A task can be updated multiple times; only the latest priority and insertion order matter.
- If a task is updated with the same priority, the most recent update defines its order relative to other same-priority tasks.
- "GET" or "PEEK" on an empty queue must print "NO TASKS".
- Priority updates must not create duplicate active tasks in the queue; only one instance of a task_name should be considered "live".

Solution Constraints:
- Use only Python standard libraries (heapq and dict are essential).
- Ensure O(log N) time complexity per ADD, GET, and PEEK operation (where N is the number of active tasks in the heap).
- The heap must only grow. No element should ever be explicitly removed or deleted from the heap except by a successful get_next_task operation returning a valid, highest-priority element.
- You must not scan or filter the heap to find outdated tasks during get_next_task or peek_next_task for cleanup; invalid tasks are handled lazily upon extraction attempts.
- Updating an existing task’s priority and its corresponding entry in the internal mapping must take O(1) time (excluding the O(log N) heap push for the new entry).
- Must use a task_map (e.g., a dictionary) to verify task validity on each pop/peek. If outdated, discard and continue.
- Insertion order must be preserved for same-priority tasks using a global counter, which is part of the tuple stored in the heap.
- The add_task method must not inspect the heap to check for duplicates.

In [ ]:
# code
"""Module to define the TaskQueue class which manages task scheduling."""

import heapq


class TaskQueue:
    """Manages a priority-based task queue."""

    def __init__(self):
        """Initialize the task queue with a heap, task map, and counter."""
        self._heap = []
        self._task_map = {}
        self._counter = 0

    def add_task(self, task_name: str, priority: int) -> None:
        """
        Add a new task or update an existing task's priority.

        Args:
            task_name (str): Name of the task.
            priority (int): Priority of the task (lower means higher priority).
        """
        self._counter += 1
        insertion_order = self._counter
        self._task_map[task_name] = (priority, insertion_order)
        heapq.heappush(self._heap, (priority, insertion_order, task_name))

    def get_next_task(self) -> str:
        """
        Remove and return the task with the highest priority.

        Returns:
            str: The task name, or "NO TASKS" if the queue is empty.
        """
        while self._heap:
            priority, order, task_name = heapq.heappop(self._heap)
            if self._task_map.get(task_name) == (priority, order):
                del self._task_map[task_name]
                return task_name
        return "NO TASKS"

    def peek_next_task(self) -> str:
        """
        Return the task with the highest priority without removing it.

        Returns:
            str: The task name, or "NO TASKS" if the queue is empty.
        """
        while self._heap:
            priority, order, task_name = self._heap[0]
            if self._task_map.get(task_name) == (priority, order):
                return task_name
            heapq.heappop(self._heap)
        return "NO TASKS"


In [ ]:
# tests

import unittest
from main import TaskQueue

class TestTaskQueue(unittest.TestCase):

    def setUp(self):
        self.queue = TaskQueue()

    def tearDown(self):
        self.queue = None

    def test_add_single_task_and_get(self):
        self.queue.add_task("task1", 5)
        result = self.queue.get_next_task()
        self.assertEqual(result, "task1")
        result = self.queue.get_next_task()
        self.assertEqual(result, "NO TASKS")

    def test_add_multiple_tasks_priority_order(self):
        self.queue.add_task("low_priority", 10)
        self.queue.add_task("high_priority", 2)
        self.queue.add_task("medium_priority", 5)
        self.assertEqual(self.queue.get_next_task(), "high_priority")
        self.assertEqual(self.queue.get_next_task(), "medium_priority")
        self.assertEqual(self.queue.get_next_task(), "low_priority")

    def test_same_priority_fifo_order(self):
        self.queue.add_task("first", 5)
        self.queue.add_task("second", 5)
        self.queue.add_task("third", 5)
        self.assertEqual(self.queue.get_next_task(), "first")
        self.assertEqual(self.queue.get_next_task(), "second")
        self.assertEqual(self.queue.get_next_task(), "third")

    def test_update_existing_task_priority(self):
        self.queue.add_task("task1", 10)
        self.queue.add_task("task2", 5)
        self.queue.add_task("task1", 1)
        self.assertEqual(self.queue.get_next_task(), "task1")
        self.assertEqual(self.queue.get_next_task(), "task2")

    def test_update_task_same_priority_new_order(self):
        self.queue.add_task("task1", 5)
        self.queue.add_task("task2", 5)
        self.queue.add_task("task3", 5)

        self.queue.add_task("task1", 5)

        self.assertEqual(self.queue.get_next_task(), "task2")
        self.assertEqual(self.queue.get_next_task(), "task3")
        self.assertEqual(self.queue.get_next_task(), "task1")

    def test_peek_without_removing(self):
        self.queue.add_task("task1", 10)
        self.queue.add_task("task2", 5)
        self.assertEqual(self.queue.peek_next_task(), "task2")
        self.assertEqual(self.queue.peek_next_task(), "task2")
        self.assertEqual(self.queue.get_next_task(), "task2")

    def test_peek_empty_queue(self):
        self.assertEqual(self.queue.peek_next_task(), "NO TASKS")

    def test_get_empty_queue(self):
        self.assertEqual(self.queue.get_next_task(), "NO TASKS")

    def test_mixed_operations_sequence(self):
        self.queue.add_task("email_processing", 5)
        self.queue.add_task("data_backup", 2)
        self.assertEqual(self.queue.get_next_task(), "data_backup")
        self.queue.add_task("system_update", 2)
        self.assertEqual(self.queue.peek_next_task(), "system_update")
        self.queue.add_task("email_processing", 1)
        self.assertEqual(self.queue.get_next_task(), "email_processing")
        self.assertEqual(self.queue.get_next_task(), "system_update")
        self.assertEqual(self.queue.peek_next_task(), "NO TASKS")

    def test_boundary_priority_values(self):
        self.queue.add_task("min_priority", 1000000)
        self.queue.add_task("max_priority", 1)
        self.assertEqual(self.queue.get_next_task(), "max_priority")
        self.assertEqual(self.queue.get_next_task(), "min_priority")

    def test_large_number_of_tasks(self):
        for i in range(1000):
            self.queue.add_task(f"task_{i}", i % 100 + 1)

        first_task = self.queue.get_next_task()
        self.assertIn("task_", first_task)


        second_task = self.queue.get_next_task()
        self.assertIn("task_", second_task)
        self.assertNotEqual(first_task, second_task)

    def test_task_name_edge_cases(self):
        long_name = "a" * 100
        self.queue.add_task(long_name, 5)
        special_name = "task_with_special-chars.123"
        self.queue.add_task(special_name, 3)
        self.assertEqual(self.queue.get_next_task(), special_name)
        self.assertEqual(self.queue.get_next_task(), long_name)

    def test_multiple_updates_same_task(self):
        self.queue.add_task("task1", 10)
        self.queue.add_task("task2", 5)
        self.queue.add_task("task1", 8)
        self.queue.add_task("task1", 3)
        self.queue.add_task("task1", 1)
        self.assertEqual(self.queue.get_next_task(), "task1")
        self.assertEqual(self.queue.get_next_task(), "task2")

    def test_complex_priority_and_fifo_interaction(self):
        self.queue.add_task("low1", 10)
        self.queue.add_task("high1", 2)
        self.queue.add_task("low2", 10)
        self.queue.add_task("high2", 2)
        self.queue.add_task("medium", 5)
        self.assertEqual(self.queue.get_next_task(), "high1")
        self.assertEqual(self.queue.get_next_task(), "high2")
        self.assertEqual(self.queue.get_next_task(), "medium")
        self.assertEqual(self.queue.get_next_task(), "low1")
        self.assertEqual(self.queue.get_next_task(), "low2")

    def test_peek_after_updates(self):
        self.queue.add_task("task1", 10)
        self.queue.add_task("task2", 5)
        self.assertEqual(self.queue.peek_next_task(), "task2")
        self.queue.add_task("task1", 1)
        self.assertEqual(self.queue.peek_next_task(), "task1")
        self.assertEqual(self.queue.get_next_task(), "task1")
        self.assertEqual(self.queue.peek_next_task(), "task2")



# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/e39f15e5-5e35-4336-92a5-e878569e5006?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67680%2Fview

#### Model code:

```python
# code generated by the model
import heapq

class TaskQueue:
    def __init__(self):
        self.heap = []
        self.entry_finder = {}  # Maps task_name to (priority, insertion_order)
        self.counter = 0  # Acts as an insertion order counter

    def add_task(self, task_name: str, priority: int) -> None:
        """Adds a task to the queue or updates its priority if it already exists."""
        if task_name in self.entry_finder:
            # Invalidate the old entry by not removing it, but ensuring lookup fails future for old priority.
            old_priority, old_order = self.entry_finder[task_name]
            if old_priority == priority:
                # <Issue>: Violates FIFO order requirement when priority is unchanged.
                return  # No need to update if the priority is unchanged
        
        # Increment the counter for insertion order
        self.counter += 1
        # Push the new task into the heap
        entry = (priority, self.counter, task_name)
        self.entry_finder[task_name] = (priority, self.counter)
        heapq.heappush(self.heap, entry)

    def get_next_task(self) -> str:
        """Removes and returns the task with the highest priority."""
        while self.heap:
            priority, order, task_name = heapq.heappop(self.heap)
            current_priority, current_order = self.entry_finder.get(task_name, (None, None))
            # Check if this task is valid
            if (current_priority, current_order) == (priority, order):
                # Found the correct task, remove from entry_finder
                del self.entry_finder[task_name]
                return task_name
            # If not valid, this means task was updated, continue popping.
        return "NO TASKS"

    def peek_next_task(self) -> str:
        """Returns the name of the task with the highest priority without removing it."""
        while self.heap:
            priority, order, task_name = self.heap[0]  # Peek at the top of the heap
            current_priority, current_order = self.entry_finder.get(task_name, (None, None))
            # Check if this task is valid
            if (current_priority, current_order) == (priority, order):
                return task_name
            # If not valid, this means task was updated, remove it.
            heapq.heappop(self.heap)
        return "NO TASKS"
```